In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 처음 사용자 입력 시 음식명으로 검색은 0, 재료명으로 검색은 1로 선택하게 하게 함.
# 0(음식명) 입력하면 음식 카테고리 리스트를, 1(재료명) 입력하면 재료 카테고리 리스트를  보여주고 다시 선택하게 함.
# 우선 0 -> 음식 카테고리 선택 -> 최종 음식명을 선택하게 됨
# 최종 음식명을 선택하게 되면 같은 이름의 요리명을 가진 레시피를 조회수와 스크랩수가 높은 순으로 3개를 추천해주는 시스템.
# 1을 선택하면 -> 재료 카테고리 선택 -> 최종 주 재료명을 선택하게 됨
# 최종 재료명을 선택하게 되면 해당 재료가 들어있는 요리를 찾아 조회수와 스크랩수가 높은 순으로 3개를 추천해주는 시스템.
# 이후 좀더 구체적으로 발전 할 계획.

In [ ]:
# 처음 사용자 입력 시 음식명으로 검색은 0, 재료명으로 검색은 1로 선택하게 함.

In [ ]:
# 레시피 데이터 불러오기
reci = pd.read_excel('./10000recipe Preprocessing_수정_3.xlsx')
reci.head(2)

,레시피제목,요리명,요리재료내용,재료,조리순서,조회수,추천수,스크랩수,요리종류별명,요리재료별명,재료2,재료2_Okt,Nouns_Okt,재료_불용어 제외
0,백전백승! 실패없는 김치볶음밥,김치볶음밥,[필수재료] 밥 1공기| 다진 양파 3큰술| 다진 김치 1/2컵| 다진 햄 3큰술 ...,"밥,다진양파,다진김치,다진햄,설탕,김칫국물,고추장,부순참깨,들기름",[조리순서]\n1.밥은 넓은 접시에 펼쳐 뜨거운 김을 날린다.밥 1공기\n2.양념장...,366065,1324,6914,밥/죽/떡,쌀,"밥, 다진양파, 다진김치, 다진햄, 설탕","[('밥', 'Noun'), (',', 'Punctuation'), ('다진', '...","['밥', '양파', '김치', '햄', '설탕']","밥,다진양파,다진김치,다진햄,설탕"
1,매일 먹는 밑반찬 메뉴로 추천!! 감자조림,감자조림,[필수재료] 감자 2개(360g)| 물 1cup [양념재료] 간장 5T| 설탕 1T...,"감자,물,간장,설탕,물엿,다진마늘,참깨,참기름",[조리순서]\n1.감자는 손질하여 한입 크기로 썰어 준비한다.감자 2개(360g)감...,478527,1102,8221,밑반찬,채소류,"감자, 간장, 설탕, 물엿, 다진마늘","[('감자', 'Noun'), (',', 'Punctuation'), ('간장', ...","['감자', '간장', '설탕', '물엿', '마늘']","감자,간장,설탕,물엿,다진마늘"


# 재료컬럼을text로 label을 요리명으로 train 한뒤 재료명을 입력하면 재료가 속해있는 요리를 추천.

In [ ]:
reci_ingre_food = reci[['재료','요리명']]

In [ ]:
reci_ingre_food.head()

,재료,요리명
0,"밥,다진양파,다진김치,다진햄,설탕,김칫국물,고추장,부순참깨,들기름",김치볶음밥
1,"감자,물,간장,설탕,물엿,다진마늘,참깨,참기름",감자조림
2,"김치,돼지고기,양파,파,김치국물,간장,고춧가루,다진마늘,다시다",김치찌개
3,"돼지고기,양파,청양고추,대파,설탕,고추장,간장,고춧가루,다진마늘,굴소스,올리고당",제육볶음
4,"고등어,무,양파,대파,청양고추,설탕,다진마늘,다진생강,된장,진간장,들기름,고추가루",고등어조림


In [ ]:
reci_ingre_food = reci_ingre_food.rename(columns={'재료':'text', '요리명':'label'})

In [ ]:
# 레이블 인코딩
label_encoder = LabelEncoder()
reci_ingre_food['label'] = label_encoder.fit_transform(reci_ingre_food['label'])

In [ ]:
# 텍스트 토큰화 패딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reci_ingre_food['text'])
sequences = tokenizer.texts_to_sequences(reci_ingre_food['text'])
padded_sequences = pad_sequences(sequences, padding='post')

In [ ]:
# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, reci_ingre_food['label'],test_size=0.2, random_state=42)

모델 구축 및 학습_LSTM사용.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [ ]:
# 모델 구성
model = Sequential()
model.add(Embedding(input_dim = len(tokenizer.word_index) +1 , output_dim = 128, input_length= padded_sequences.shape[1]))
model.add(LSTM(128))
model.add(Dense(4, activation = 'softmax'))

In [ ]:
# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 레시피 데이터 불러오기
reci = pd.read_excel('./10000recipe Preprocessing_수정_3.xlsx')

# 레이블 인코딩
label_encoder = LabelEncoder()
reci_ingre_food['label'] = label_encoder.fit_transform(reci_ingre_food['label'])


# 텍스트 토큰화 패딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reci_ingre_food['text'])
sequences = tokenizer.texts_to_sequences(reci_ingre_food['text'])
padded_sequences = pad_sequences(sequences, padding='post')


# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, reci_ingre_food['label'],test_size=0.2, random_state=42)


# 모델 구성
model = Sequential()
model.add(Embedding(input_dim = len(tokenizer.word_index) +1 , output_dim = 128, input_length= padded_sequences.shape[1]))
model.add(LSTM(128))
model.add(Dense(4, activation = 'softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(X_train, y_train, epochs=200, validation_data= (X_test, y_test))

Epoch 1/200
38/38 [==============================] - 4s 30ms/step - loss: nan - accuracy: 0.0117 - val_loss: nan - val_accuracy: 0.0133
Epoch 2/200
38/38 [==============================] - 1s 14ms/step - loss: nan - accuracy: 0.0117 - val_loss: nan - val_accuracy: 0.0133
Epoch 3/200
38/38 [==============================] - 1s 15ms/step - loss: nan - accuracy: 0.0117 - val_loss: nan - val_accuracy: 0.0133
Epoch 4/200
38/38 [==============================] - 1s 14ms/step - loss: nan - accuracy: 0.0117 - val_loss: nan - val_accuracy: 0.0133
Epoch 5/200
38/38 [==============================] - 1s 14ms/step - loss: nan - accuracy: 0.0117 - val_loss: nan - val_accuracy: 0.0133
Epoch 6/200
38/38 [==============================] - 1s 14ms/step - loss: nan - accuracy: 0.0117 - val_loss: nan - val_accuracy: 0.0133
Epoch 7/200
38/38 [==============================] - 1s 14ms/step - loss: nan - accuracy: 0.0117 - val_loss: nan - val_accuracy: 0.0133
Epoch 8/200
38/38 [=============================

In [ ]:
# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy : {accuracy}')

10/10 [==============================] - 0s 18ms/step - loss: nan - accuracy: 0.0133
Test Accuracy : 0.013333333656191826


모델평가의 정확도가 0.013으로 너무 낮게 나와 모델 성능을 향상시키기 위한 여러가지 방법 고려.
1. 데이터셋이 너무 작을 경우 많은 데이터를 수집하거나 데이터 증강기법 사용.
2. 데이터 균형 확인
- 클래스 간의 불균형이 있는지 확인.
- 불균형 데이터 처리(오버샘플링, 언더샘플링, SMOTE)
3. 모델 개선
- 다양한 모델 시도( CNN, GRU, BERT, GPT 등)
- 하이퍼라마미터 튜닝
  -> 임베딩 차원: 임베딩 레이어 차원 조정
  -> LSTM 유닛 수 : LSTM 유닛 수 조정
  -> 드롭아웃 : 드롭아웃 레이어를 추가하여 오버피팅 방지
  -> 학습률 : 옵티마이저의 학습률 조정
4. 정교한 모델링
- 전이학습
 -> BERT: 사전 학습된 BERT 모델을 활용해 텍스트 분류 문제에 적용
 -> GPT:  사전 학습된 GPT 모델 활용
5. 모델 평가 방법 개선
- 교차검증 : 데이터를 여러개의 폴드로 나누어 교차검증 수행. 모델의 일반화 성능 평가


In [ ]:
#  1.데이터 균형 확인
reci_ingre_food['label'].value_counts()

label
잡채       72
두부조림     72
닭볶음탕     65
제육볶음     58
순두부찌개    49
         ..
열무국수      1
숙주나물      1
삼겹살       1
순대        1
보쌈        1
Name: count, Length: 126, dtype: int64

In [ ]:
# 클래스 불균형 문제 해결_SMOTE 사용.
# -> Expected n_neighbors <= n_samples_fit, but n_neighbors = 2, n_samples_fit = 1, n_samples = 1 오류.

In [ ]:
# pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
reci_ingre_food = reci_ingre_food.rename(columns={'재료':'text', '요리명':'label'})
# 레이블 인코딩
label_encoder = LabelEncoder()
reci_ingre_food['label'] = label_encoder.fit_transform(reci_ingre_food['label'])

# 텍스트 토큰화 패딩
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reci_ingre_food['text'])
sequences = tokenizer.texts_to_sequences(reci_ingre_food['text'])
padded_sequences = pad_sequences(sequences, padding='post')

# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, reci_ingre_food['label'],test_size=0.2, random_state=42)

In [ ]:
# #SMOTE 적용
# from imblearn.over_sampling import SMOTE
# from collections import Counter

In [ ]:
# # 소수 클래스의 데이터 포인트 수 확인
# counter = Counter(y_train)
# print(counter)

Counter({98: 59, 43: 59, 35: 53, 102: 45, 47: 40, 86: 37, 76: 33, 114: 29, 56: 29, 7: 26, 53: 25, 24: 25, 26: 23, 22: 22, 113: 22, 41: 22, 13: 18, 90: 17, 14: 16, 6: 16, 116: 15, 59: 15, 33: 15, 0: 14, 54: 14, 21: 14, 5: 14, 87: 14, 46: 14, 60: 14, 89: 13, 17: 13, 39: 12, 58: 12, 42: 12, 40: 11, 81: 11, 95: 11, 97: 11, 91: 10, 8: 10, 108: 10, 115: 9, 27: 9, 12: 9, 94: 9, 9: 9, 111: 9, 1: 8, 92: 7, 107: 7, 45: 7, 93: 7, 106: 7, 2: 7, 49: 7, 48: 6, 51: 6, 64: 6, 3: 6, 61: 6, 104: 6, 101: 5, 67: 5, 36: 5, 72: 5, 121: 5, 23: 5, 44: 5, 68: 4, 11: 4, 125: 4, 110: 4, 73: 4, 31: 4, 25: 4, 123: 4, 29: 4, 78: 4, 55: 4, 16: 3, 79: 3, 120: 3, 84: 3, 34: 3, 65: 3, 30: 3, 77: 3, 63: 3, 112: 3, 37: 3, 28: 2, 109: 2, 38: 2, 20: 2, 19: 2, 4: 2, 85: 2, 96: 2, 32: 2, 70: 2, 105: 2, 69: 2, 119: 2, 18: 2, 52: 1, 15: 1, 57: 1, 83: 1, 118: 1, 74: 1, 88: 1, 50: 1, 99: 1, 103: 1, 82: 1, 80: 1, 71: 1, 122: 1, 10: 1, 66: 1, 62: 1})


In [ ]:
# #SMOTE 적용
# n_samples_fit = min(counter.values()) -1  # 가장 적은 클래스의 샘플 수 -1
# n_neighbors = max(1, n_samples_fit) # n_neighbors는 최소 1 이상이어야 함.

In [ ]:
# smote = SMOTE(random_state =42, k_neighbors=n_neighbors)
# X_resample, y_resample = smote.fit_resample(X_train, y_train)

In [ ]:
# 클래스 불균형 문제 해결_RandomOverSampler 사용.

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
# # RandomoverSampler 사용.
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [ ]:
# pip install -U imbalanced-learn

In [ ]:
from keras.regularizers import l2
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
#모델 구성
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=padded_sequences.shape[1]))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(4, activation='softmax'))

In [ ]:
from keras.optimizers import Adam

In [ ]:
# 모델 컴파일
optimizer = Adam(learning_rate = 0.001) # 학습률 설정
model.compile(optimizer= optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 모델 학습
model.fit(X_resampled, y_resampled, epochs=50, validation_data= (X_test, y_test))

Epoch 1/50
225/225 [==============================] - 10s 36ms/step - loss: nan - accuracy: 0.0082 - val_loss: nan - val_accuracy: 0.0133
Epoch 2/50
225/225 [==============================] - 7s 33ms/step - loss: nan - accuracy: 0.0082 - val_loss: nan - val_accuracy: 0.0133
Epoch 3/50
225/225 [==============================] - 7s 33ms/step - loss: nan - accuracy: 0.0082 - val_loss: nan - val_accuracy: 0.0133
Epoch 4/50
225/225 [==============================] - 7s 33ms/step - loss: nan - accuracy: 0.0082 - val_loss: nan - val_accuracy: 0.0133
Epoch 5/50
225/225 [==============================] - 7s 33ms/step - loss: nan - accuracy: 0.0082 - val_loss: nan - val_accuracy: 0.0133
Epoch 6/50
225/225 [==============================] - 8s 34ms/step - loss: nan - accuracy: 0.0082 - val_loss: nan - val_accuracy: 0.0133
Epoch 7/50
225/225 [==============================] - 7s 32ms/step - loss: nan - accuracy: 0.0082 - val_loss: nan - val_accuracy: 0.0133
Epoch 8/50
225/225 [====================

In [ ]:
# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy : {accuracy}')

10/10 [==============================] - 0s 14ms/step - loss: nan - accuracy: 0.0133
Test Accuracy : 0.013333333656191826


In [ ]:
reci['요리종류별명'].unique()

array(['밥/죽/떡', '밑반찬', '찌개', '메인반찬', '면', '김치/젓갈/장류', '국/탕', '분식', '디저트',
       '피자'], dtype=object)

In [ ]:
reci.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   레시피제목      1499 non-null   object
 1   요리명        1499 non-null   object
 2   요리재료내용     1499 non-null   object
 3   재료         1499 non-null   object
 4   조리순서       1499 non-null   object
 5   조회수        1499 non-null   int64 
 6   추천수        1499 non-null   int64 
 7   스크랩수       1499 non-null   int64 
 8   요리종류별명     1499 non-null   object
 9   요리재료별명     1499 non-null   object
 10  재료2        1499 non-null   object
 11  재료2_Okt    1499 non-null   object
 12  Nouns_Okt  1499 non-null   object
 13  재료_불용어 제외  1499 non-null   object
dtypes: int64(3), object(11)
memory usage: 164.1+ KB


In [ ]:
# 전체 함수

#치환함수 정의
def normalize_word(word):
    replacements = {
        "달걀" :"계란",
        "계란": "계란",
        "오뎅":"어묵",
        "어묵":"어묵",
        "무우":"무",
        "무":"무",
        "대파":"파",
        "파":"파",
        "씨레기":"시래기",
        "시래기":"시래기"
    }
    return replacements.get(word,word)


# 사용자의 입력을 받는 함수 정의
def get_user_choice():
    choice = input("쿠킹파트너 : 음식명 검색은 0, 재료명 검색은 1을 입력해 주세요~^^")
    return choice


# 음식 카테고리 1 선택 함수 정의
def show_categories(df):
    categories = df['요리종류별명'].unique()
    for idx, category in enumerate(categories, start=1):
        print(f"{idx}.{category}")
    return categories


# 음식 카테고리 1 내 요리명 선택 함수 정의
def show_recipes_by_category(df, category):
    filterd_df = df[df['요리종류별명'] == category]
    recipes = filterd_df['요리명'].unique()
    for idx, recipe in enumerate(recipes, start=1):
        print(f"{idx}.{recipe}")
    return recipes


# 요리명 선택 후 추천 요리 방법 함수 정의
def recommend_recipes(df, recipe_name):
    filtered_df = df[df['요리명'].str.contains(recipe_name)]
    sorted_df = filtered_df.sort_values(by=['조회수', '스크랩수'], ascending=[False, False])
    top_recipes = sorted_df.head(3)

    responses = []
    for i, match in enumerate(top_recipes.iterrows(), start=1):  # 최대 3개까지 출력
        response = f"\n<{i}. {match[1]['레시피제목']}>\n\n재료:\n{match[1]['요리재료내용']}\n\n조리순서:\n{match[1]['조리순서']}"
        if i == 1:
            response = f"\n 쿠킹파트너 : {match[1]['요리명']} 레시피를 알려드리겠습니다.\n" + response
        responses.append(response)

    return '\n'.join(responses)



#재료명을 기반으로 요리 레시피 추천 함수 정의
def recommend_recipes_by_ingredients(df, ingredients):
    normalized_ingredients = [normalize_word(ingredient) for ingredient in ingredients]

    # 각 레시피의 재료 목록이 모두 입력된 재료와 일치하는지 확인
    filtered_df = df[df['재료_불용어 제외'].apply(lambda x: all(item in x for item in normalized_ingredients))]
    sorted_df = filtered_df.sort_values(by=['조회수','스크랩수'], ascending =[False, False])
    top_recipes = sorted_df.head(3)

    if top_recipes.empty:
        return "일치하는 레시피가 없습니다."

    responses =[]
    for i , match in enumerate(top_recipes.iterrows(), start=1): #최대 3개까지 출력
        response =  f"\n<{i}. {match[1]['레시피제목']}>\n\n재료:\n{match[1]['요리재료내용']}\n\n조리순서:\n{match[1]['조리순서']}"
        if i == 1:
            response = f"\n 쿠킹파트너 : {match[1]['요리명']} 레시피를 알려드리겠습니다.\n" + response
        responses.append(response)

    return '\n'.join(responses)



# 메인 실행 함수
def main():
    user_choice = get_user_choice()

    if user_choice == '0':
        categories = show_categories(reci)
        category_choice = input("쿠킹파트너 : 1~10까지의 숫자를 입력하거나 카테고리명을 입력하세요: ")

        if category_choice.isdigit():
            category_choice = int(category_choice)
            if 1 <= category_choice <= len(categories):
                selected_category = categories[category_choice -1]
            else:
                print("쿠킹파트너 : 다시 선택해주세요~")
                return
        else:
            if category_choice in categories:
                selected_category = category_choice
            else:
                print("쿠킹파트너: 다시 선택해주세요~")
                return

        recipes = show_recipes_by_category(reci, selected_category)
        recipe_choice = input( "쿠킹파트너: 숫자를 입력하거나 요리명을 입력하세요: ")

        if recipe_choice.isdigit():
            recipe_choice = int(recipe_choice)
            if 1 <= recipe_choice <= len(recipes):
                selected_recipe = recipes[recipe_choice -1]
            else:
                print("쿠킹 파트너: 다시 선택해주세요~")
                return

        else:
            if recipe_choice in recipes:
                selected_recipe =recipe_choice
            else:
                print("쿠킹 파트너: 다시 선택해주세요~")
                return


        print(recommend_recipes(reci, selected_recipe))


    elif user_choice == '1':
        ingredients = input("쿠킹파트너: 재료를 3가지 입력하세요 (쉼표로 구분해주세요^^): ").split(',')
        ingredients =[normalize_word(ingredient.strip()) for ingredient in ingredients]
        if len(ingredients) !=3:
            print("쿠킹 파트너 : 재료를 정확히 3가지 입력해주세요~")
            return
        print(recommend_recipes_by_ingredients(reci, ingredients))

    else:
        print("쿠킹 파너트 : 0과 1중에 입력해주세요")

if __name__ == "__main__":
    main()

쿠킹 파트너 : 재료를 정확히 3가지 입력해주세요~


# 재료 추천을 자연어 처리를 활용한 방법으로 수정해보기.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# nltk 데이터 다운로드
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tjoeun\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tjoeun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tjoeun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# 재료
# 전체 함수

# 불용어 및 특수문자 제거, 단어를 소문자로 변환 및 표준형으로 변환하여 텍스트를 정규화 하는 함수
def preprocess_text(text):
    # WordnetLemmatizer 초기화
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_text = [lemmatizer.lemmatize(word.lower()) for word in word_tokens if word.isalnum() and word.lower() not in stop_words]
    return ' '.join(filtered_text)



#치환함수 정의
def normalize_word(word):
    replacements = {
        "달걀" :"계란",
        "계란": "계란",
        "오뎅":"어묵",
        "어묵":"어묵",
        "무우":"무",
        "무":"무",
        "대파":"파",
        "파":"파",
        "씨레기":"시래기",
        "시래기":"시래기"
    }
    return replacements.get(word,word)


# 사용자의 입력을 받는 함수 정의
def get_user_choice():
    choice = input("쿠킹파트너 : 음식명 검색은 0, 재료명 검색은 1을 입력해 주세요~^^")
    return choice


# 음식 카테고리 1 선택 함수 정의
def show_categories(df):
    categories = df['요리종류별명'].unique()
    for idx, category in enumerate(categories, start=1):
        print(f"{idx}.{category}")
    return categories


# 음식 카테고리 1 내 요리명 선택 함수 정의
def show_recipes_by_category(df, category):
    filterd_df = df[df['요리종류별명'] == category]
    recipes = filterd_df['요리명'].unique()
    for idx, recipe in enumerate(recipes, start=1):
        print(f"{idx}.{recipe}")
    return recipes


# 요리명 선택 후 추천 요리 방법 함수 정의
def recommend_recipes(df, recipe_name):
    filtered_df = df[df['요리명'].str.contains(recipe_name)]
    sorted_df = filtered_df.sort_values(by=['조회수', '스크랩수'], ascending=[False, False])
    top_recipes = sorted_df.head(3)

    responses = []
    for i, match in enumerate(top_recipes.iterrows(), start=1):  # 최대 3개까지 출력
        response = f"\n<{i}. {match[1]['레시피제목']}>\n\n재료:\n{match[1]['요리재료내용']}\n\n조리순서:\n{match[1]['조리순서']}"
        if i == 1:
            response = f"\n 쿠킹파트너 : {match[1]['요리명']} 레시피를 알려드리겠습니다.\n" + response
        responses.append(response)

    return '\n'.join(responses)



#재료명을 기반으로 요리 레시피 추천 함수 정의
def recommend_recipes_by_ingredients_nlp(df, ingredients):
    # 재료 리스트를 문자열로 반환
    query = ' '.join(ingredients)

    #레시피 데이터 전처리
    df['재료_전처리'] = df['요리재료내용'].apply(preprocess_text)

    #TF-IDF 벡터화
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df['재료_전처리'])

    # 입력된 재료에 대한 TF-IDF 벡터
    query_vector = vectorizer.transform([preprocess_text(query)])

    # 코사인 유사도 계산
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()

    # 유사도가 높은 순서로 정렬된 인덱스
    related_recipe_indices = cosine_similarities.argsort()[::-1]

    #가장 관련성 높은 레시피 3개 선택
    top_related_indices = related_recipe_indices[:3]

    responses =[]
    for i , idx in enumerate(top_related_indices, start=1): #최대 3개까지 출력
        response =  f"\n<{i}. {df.iloc[idx]['레시피제목']}>\n\n재료:\n{df.iloc[idx]['요리재료내용']}\n\n조리순서:\n{df.iloc[idx]['조리순서']}"
        if i == 1:
            response = f"\n 쿠킹파트너 : {df.iloc[idx]['요리명']} 레시피를 알려드리겠습니다.\n" + response
        responses.append(response)

    return '\n'.join(responses)



# 메인 실행 함수
def main():
    user_choice = get_user_choice()

    if user_choice == '0':
        categories = show_categories(reci)
        category_choice = input("쿠킹파트너 : 1~10까지의 숫자를 입력하거나 카테고리명을 입력하세요: ")

        if category_choice.isdigit():
            category_choice = int(category_choice)
            if 1 <= category_choice <= len(categories):
                selected_category = categories[category_choice -1]
            else:
                print("쿠킹파트너 : 다시 선택해주세요~")
                return
        else:
            if category_choice in categories:
                selected_category = category_choice
            else:
                print("쿠킹파트너: 다시 선택해주세요~")
                return

        recipes = show_recipes_by_category(reci, selected_category)
        recipe_choice = input( "쿠킹파트너: 숫자를 입력하거나 요리명을 입력하세요: ")

        if recipe_choice.isdigit():
            recipe_choice = int(recipe_choice)
            if 1 <= recipe_choice <= len(recipes):
                selected_recipe = recipes[recipe_choice -1]
            else:
                print("쿠킹 파트너: 다시 선택해주세요~")
                return

        else:
            if recipe_choice in recipes:
                selected_recipe =recipe_choice
            else:
                print("쿠킹 파트너: 다시 선택해주세요~")
                return


        print(recommend_recipes(reci, selected_recipe))


    elif user_choice == '1':
        ingredients = input("쿠킹파트너: 재료를 3가지 입력하세요 (쉼표로 구분해주세요^^): ").split(',')
        if len(ingredients) !=3:
            print("쿠킹 파트너 : 재료를 정확히 3가지 입력해주세요~")
            return
        print(recommend_recipes_by_ingredients_nlp(reci, ingredients))

    else:
        print("쿠킹 파너트 : 0과 1중에 입력해주세요")

if __name__ == "__main__":
    main()


 쿠킹파트너 : 김치찜 레시피를 알려드리겠습니다.

<1. 【단시간에 깊은 맛이 일품인 김치찜 만드는 방법】>

재료:
[재료] 김치 1/4포기| 돼지고기 200g| 대파 1/2줄기 [양념] 참기름 2T| 설탕 1T| 맛술 3T

조리순서:
[조리순서]
1.김치(1/4포기)는 꼬다리(?)를 잘라 궁중팬에 넓게 펼쳐 주세요~ (2인분기준 한끼 드실 양입니다^^)
2.참기름 두스푼 넣고 중불에서 볶아 주세용^~^ (김치찜인데 볶는게 생소 하실텐데요~진짜 자신 있게 말씀 드릴수 있어요~☞☜이대로만 하시믄 완전 맛있어요~!ㅎㅎ)
3.돼지고기(200g)는 핏물을 빼서 맛술 3스푼 넣어 잠시 두셨다가 이때 같이 넣고 볶아 주세용^~^
4.설탕도 한스푼 넣어 주시구요~~
5.타지않게 중약불에서 고기 겉면이 익을때 까지 볶으시다가
6.김치가 잠길정도의 물을 넣고 쌘불로 끓여 주세요~
7.그사이 대파 반줄기도 썰어 주시구요~
8.요렇게 국물이 졸아 들면
9.다시 물을 넣고 끓여 주세요^~^ (계속 쌘불 입니다~ 잠시잠시 끓을 동안 저는 소세지도 같이 구웠어용~ㅎㅎ 완전 편한 방법이라 다른거 하시면서 대충 물만 총3번 채워 주심 되셔요^~^ )
10.요렇게 또 국물이 졸아 들면
11.마지막으로 물을 넣어 대파(반줄기)올려서 끓여 주시면 끝이에요~!!
12.요렇게 국물이 자작하게 있어야 더맛나죵^~^ 마지막엔 완전 졸이시지 마시고 국물을 자작하게 남겨 두셔용~~^~^
13.쭉쭉 찢어서 밥위에 올려먹고 고기에 싸서 먹고 밥에 돌돌 말아먹고 김치랑 고기 탑(?)으로 쌓아서 먹고.. 별다른 재료 없이 초간단으로 오래 푹 끓인거 같이 깊은 맛을 느끼실수 있으실꺼에요~!!!! (너무 맛있어서 오늘도 다이어트는 물건너 갔어요ㅠ)
14.편의점표 볶음김치 만드는 방법 바로가기☞ @6866485

<2. 김치찌개 맛있게 끓이는 법>

재료:
[재료] 돼지고기 앞다리살 200g| 익은 김치 2컵| 대파 1/2대| 청양고추 1개| 두부 1/4모| 고춧가루 1.5T [양념] 김치국물 1/2컵| 다시